In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import sklearn
import seaborn as sns

df = pd.read_csv(r"C:\Users\seanr\Desktop\fake_news_train.csv")
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [2]:
df=df.dropna()

In [3]:
## Get the Independent Features

x=df.drop('label',axis=1)

## Get the Dependent features
y=df['label']

In [4]:
x.reset_index(inplace=True)

In [5]:
import tensorflow as tf

In [6]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense

# TEXT PREPRCOESSING , ONE HOT ENCODING 

In [7]:
import nltk
import re
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

In [8]:
lemmatizer=WordNetLemmatizer()

corpus=[]

for i in range(len(x)):
    review=re.sub("[^a-zA-Z]"," ",x["title"][i])
    
    review=review.lower()
    
    review=review.split()
    
    review=[lemmatizer.lemmatize(word) for word in review if word not in set(stopwords.words("english"))]
    
    review=" ".join(review)
    
    corpus.append(review)
    
print(corpus)
    

['house dem aide even see comey letter jason chaffetz tweeted', 'flynn hillary clinton big woman campus breitbart', 'truth might get fired', 'civilian killed single u airstrike identified', 'iranian woman jailed fictional unpublished story woman stoned death adultery', 'jackie mason hollywood would love trump bombed north korea lack trans bathroom exclusive video breitbart', 'beno hamon win french socialist party presidential nomination new york time', 'back channel plan ukraine russia courtesy trump associate new york time', 'obama organizing action partner soros linked indivisible disrupt trump agenda', 'bbc comedy sketch real housewife isi cause outrage', 'russian researcher discover secret nazi military base treasure hunter arctic photo', 'u official see link trump russia', 'yes paid government troll social medium blog forum website', 'major league soccer argentine find home success new york time', 'well fargo chief abruptly step new york time', 'anonymous donor pay million release

In [9]:
### Vocabulary size
voc_size=5000


onehot_repr=[one_hot(words,voc_size)for words in corpus] 
onehot_repr[0]


[4060, 3888, 4830, 3217, 4846, 1186, 2306, 2770, 1147, 1885]

In [10]:
sent_length=20
embedded_docs=pad_sequences(onehot_repr,padding='pre',maxlen=sent_length)
print(embedded_docs)

[[   0    0    0 ... 2770 1147 1885]
 [   0    0    0 ... 2202 1149 4510]
 [   0    0    0 ... 1863 4838 1440]
 ...
 [   0    0    0 ... 2436 3096 3887]
 [   0    0    0 ... 4559 4746 2622]
 [   0    0    0 ... 1535 3168 4243]]


In [11]:
embedded_docs.shape

(18285, 20)

In [12]:
embedded_docs[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0, 4060,
       3888, 4830, 3217, 4846, 1186, 2306, 2770, 1147, 1885])

# CREATING A LSTM MODEL

In [24]:
## Creating model
embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(LSTM(100))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 20, 40)            200000    
                                                                 
 lstm (LSTM)                 (None, 100)               56400     
                                                                 
 dense (Dense)               (None, 1)                 101       
                                                                 
Total params: 256,501
Trainable params: 256,501
Non-trainable params: 0
_________________________________________________________________
None


In [25]:
import numpy as np
x_final=np.array(embedded_docs)
y_final=np.array(y)

In [26]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x_final, y_final, test_size=0.33, random_state=42)


In [28]:
### Finally Training
model.fit(x_train,y_train,validation_data=(x_test,y_test),epochs=10,batch_size=64)

Epoch 1/10
192/192 [==============================] - 5s 18ms/step - loss: 0.3404 - accuracy: 0.8476 - val_loss: 0.2030 - val_accuracy: 0.9147
Epoch 2/10
192/192 [==============================] - 3s 16ms/step - loss: 0.1427 - accuracy: 0.9435 - val_loss: 0.2001 - val_accuracy: 0.9251
Epoch 3/10
192/192 [==============================] - 3s 15ms/step - loss: 0.0923 - accuracy: 0.9672 - val_loss: 0.2296 - val_accuracy: 0.9215
Epoch 4/10
192/192 [==============================] - 3s 15ms/step - loss: 0.0593 - accuracy: 0.9794 - val_loss: 0.2450 - val_accuracy: 0.9198
Epoch 5/10
192/192 [==============================] - 3s 16ms/step - loss: 0.0376 - accuracy: 0.9878 - val_loss: 0.3702 - val_accuracy: 0.9137
Epoch 6/10
192/192 [==============================] - 3s 15ms/step - loss: 0.0188 - accuracy: 0.9949 - val_loss: 0.4129 - val_accuracy: 0.9185
Epoch 7/10
192/192 [==============================] - 3s 16ms/step - loss: 0.0103 - accuracy: 0.9972 - val_loss: 0.5077 - val_accuracy: 0.9176